In [1]:
import pandas as pd
import numpy as np
from typing import Dict, Tuple
from datetime import datetime
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt

plt.rc('font', family='NanumGothic')

In [2]:
time_table = pd.read_csv('./input/seoul_metro.csv', encoding="cp949", low_memory=False)

In [3]:
time_table[(time_table['방향'] == 'UP')dlc & (time_table['주중주말'] == 'DAY') & (time_table['급행여부'] == 0)].sample(10)

,고유번호,호선,역사코드,역사명,주중주말,방향,급행여부,열차코드,열차도착시간,열차출발시간,출발역,도착역
211033,211034,4,422,동대문역사문화공원,DAY,UP,0,4664K,19:44:30,19:45:00,오이도,불암산
265715,265716,5,2534,광화문,DAY,UP,0,5592,13:57:40,13:58:10,마천,방화
319260,319261,6,2647,화랑대,DAY,UP,0,6220,16:55:20,16:55:50,봉화산,응암
160864,160865,3,319,종로3가,DAY,UP,0,3040K,06:45:30,06:46:00,오금,대화
218377,218378,4,428,삼각지,DAY,UP,0,4034,07:59:00,07:59:30,사당,불암산
329006,329007,7,2719,태릉입구,DAY,UP,0,7356,21:55:10,21:55:40,석남,도봉산
158794,158795,3,317,경복궁,DAY,UP,0,3216,14:14:30,14:15:00,오금,대화
260504,260505,5,2529,마포,DAY,UP,0,5662,20:38:00,20:38:30,마천,방화
84223,84224,1,1907,가능,DAY,UP,0,K18,08:22:30,08:23:00,인천,동두천
166365,166366,3,324,금호,DAY,UP,0,3310,18:29:00,18:29:30,오금,구파발


In [4]:
time_table.tail(20)

,고유번호,호선,역사코드,역사명,주중주말,방향,급행여부,열차코드,열차도착시간,열차출발시간,출발역,도착역
423413,423414,9,4138,중앙보훈병원,SAT,UP,1,E9646,19:38:30,NaN,김포공항,중앙보훈병원
423414,423415,9,4138,중앙보훈병원,SAT,UP,1,E9648,19:49:30,NaN,김포공항,중앙보훈병원
423415,423416,9,4138,중앙보훈병원,SAT,UP,1,E9650,20:00:30,NaN,김포공항,중앙보훈병원
423416,423417,9,4138,중앙보훈병원,SAT,UP,1,E9652,20:11:30,NaN,김포공항,중앙보훈병원
423417,423418,9,4138,중앙보훈병원,SAT,UP,1,E9654,20:22:30,NaN,김포공항,중앙보훈병원
423418,423419,9,4138,중앙보훈병원,SAT,UP,1,E9656,20:33:30,NaN,김포공항,중앙보훈병원
423419,423420,9,4138,중앙보훈병원,SAT,UP,1,E9658,20:44:30,NaN,김포공항,중앙보훈병원
423420,423421,9,4138,중앙보훈병원,SAT,UP,1,E9660,20:55:30,NaN,김포공항,중앙보훈병원
423421,423422,9,4138,중앙보훈병원,SAT,UP,1,E9662,21:06:30,NaN,김포공항,중앙보훈병원
423422,423423,9,4138,중앙보훈병원,SAT,UP,1,E9664,21:17:30,NaN,김포공항,중앙보훈병원


In [5]:
line_2 = time_table[time_table['호선'] == 2].copy()

In [6]:
line_2.sample(10)

,고유번호,호선,역사코드,역사명,주중주말,방향,급행여부,열차코드,열차도착시간,열차출발시간,출발역,도착역
103167,103168,2,211,성수,END,IN,0,2207,NaN,13:35:00,성수,성수
126537,126538,2,229,봉천,DAY,OUT,0,2010,05:31:00,05:31:30,서울대입구,성수
129662,129663,2,231,신대방,END,OUT,0,2380,21:05:00,21:05:30,성수,성수
137070,137071,2,236,영등포구청,SAT,OUT,0,2364,20:32:30,20:33:00,성수,성수
146029,146030,2,244,용답,DAY,UP,0,1588,12:55:30,12:56:00,신설동,성수
147819,147820,2,247,도림천,DAY,DOWN,0,5541,09:23:00,09:23:30,까치산,신도림
144716,144717,2,243,충정로,DAY,IN,0,2239,12:41:30,12:42:00,성수,성수
94589,94590,2,205,동대문역사문화공원,DAY,OUT,0,2368,19:30:00,19:31:00,성수,성수
124592,124593,2,227,낙성대,END,OUT,0,2342,19:11:30,19:12:00,성수,성수
127625,127626,2,230,신림,DAY,IN,0,2141,09:39:00,09:39:30,성수,성수


In [7]:
time_table[(time_table["열차도착시간"].str.startswith("15", na=False)) &
                    (time_table['호선'] ==5) &
                    (time_table["급행여부"] == 0)].groupby(["출발역", "도착역"]).agg({
    "열차코드": "first",
    "열차출발시간": "first",
    "방향": "first",
}).reset_index()

,출발역,도착역,열차코드,열차출발시간,방향
0,마천,방화,5596,15:06:30,UP
1,방화,마천,5631,15:04:00,DOWN
2,방화,하남검단산,5117,15:10:30,DOWN
3,하남검단산,방화,5118,15:13:00,UP


# Line 1

In [8]:
# 인천 -> 연천
line_1_main = time_table[(time_table["열차코드"] == "K112") & (time_table["주중주말"] == "DAY")].sort_values("열차출발시간")

In [9]:
# 구로 -> 신창, 광명 X
line_1_junction_at_guro_main = time_table[(time_table["열차코드"] == "K665") & (time_table["주중주말"] == "DAY")].sort_values("열차출발시간")

In [10]:
# 광명 -> 영등포
line_1_junction_gwangmyeong_to_yungdeonpo = time_table[(time_table["열차코드"] == "K7022") & (time_table["주중주말"] == "DAY")].sort_values("열차출발시간").reset_index(drop=True)

# 광명 -> 금천구청으로 cut
line_1_junction_at_geumcheon_gu_office = line_1_junction_gwangmyeong_to_yungdeonpo.iloc[:2]

In [11]:
# 서동탄 -> 구로
line_1_junction_seodongtan_to_guro = time_table[(time_table["열차코드"] == "K462") & (time_table["주중주말"] == "DAY")].sort_values("열차출발시간").reset_index(drop=True)
line_1_junction_at_byeongjeom = line_1_junction_seodongtan_to_guro.iloc[:2]

# Line 2

In [12]:
# 내선
line_2_in = time_table[(time_table["열차코드"] == "2293") & (time_table["주중주말"] == "DAY")].sort_values("열차출발시간")

#외선
# line_2_out = time_table[(time_table["열차코드"] == "2252") & (time_table["주중주말"] == "DAY") & (time_table["방향"] == "OUT")].sort_values("열차출발시간")

In [13]:
# 까치산 -> 신도림
line_2_junction_kachisan = time_table[(time_table["열차코드"] == "5609") & (time_table["호선"] == 2) & (time_table["주중주말"] == "DAY")].sort_values("열차출발시간")

In [14]:
# 신설동 -> 성수 +) 5631은 5호선과 공유 코드 -> 2호선 필터 필요!!
line_2_junction_sinsuldong = time_table[(time_table["열차코드"] == "1612") & (time_table["호선"] == 2) & (time_table["주중주말"] == "DAY")].sort_values("열차출발시간")

# Line 3

In [15]:
line_3_main = time_table[(time_table["열차코드"] == "3224") & (time_table["주중주말"] == "DAY")].sort_values("열차출발시간")

# Line 4

In [43]:
# 진접 -> 사당
line_4_split_1 = time_table[(time_table["열차코드"] == "S4085") & (time_table["주중주말"] == "DAY")].sort_values("열차출발시간")
print(line_4_split_1.tail(1))

          고유번호  호선  역사코드 역사명 주중주말    방향  급행여부   열차코드    열차도착시간 열차출발시간 출발역 도착역
224507  224508   4   433  사당  DAY  DOWN     0  S4085  13:08:00    NaN  진접  사당


In [42]:
# 사당 -> 오이도
line_4_split_2 = time_table[(time_table["열차코드"] == "S4507") & (time_table["주중주말"] == "DAY")].sort_values("열차출발시간")
print(line_4_split_2.head(1))


          고유번호  호선  역사코드 역사명 주중주말    방향  급행여부   열차코드 열차도착시간    열차출발시간 출발역  도착역
224579  224580   4   433  사당  DAY  DOWN     0  S4507    NaN  05:50:00  사당  오이도


# Line 5

In [18]:
# 방화 -> 하남검단산
line_5_main = time_table[(time_table["열차코드"] == "5117") & (time_table["주중주말"] == "DAY")].sort_values("열차출발시간")

# 방화 -> 마천 +) 5631은 2호선과 공유 코드 -> 5호선 필터 필요!!
line_5_junction_raw = time_table[(time_table["열차코드"] == "5631") & (time_table["호선"] == 5) & (time_table["주중주말"] == "DAY")].sort_values("열차출발시간").reset_index(drop=True)
start_idx = line_5_junction_raw[line_5_junction_raw["역사명"] == "강동"].index[0]
end_idx = line_5_junction_raw[line_5_junction_raw["역사명"] == "마천"].index[0]

# => 강동 -> 마천으로 cut
line_5_junction = line_5_junction_raw.iloc[start_idx:end_idx + 1].reset_index(drop=True)

# Travel Time Extraction & Graph Construction

In [19]:
def extract_train_times(train_df: pd.DataFrame) -> Dict[Tuple[str, str], float]:
    times = {}

    for i in range(len(train_df) - 1):
        current = train_df.iloc[i]
        next_st = train_df.iloc[i + 1]

        if current['역사명'] == next_st['역사명']:
            continue

        try:
            depart = datetime.strptime(current['열차출발시간'], '%H:%M:%S')
            arrive = datetime.strptime(next_st['열차도착시간'], '%H:%M:%S')

            if arrive >= depart:
                minutes = (arrive - depart).seconds / 60
            else:
                minutes = ((arrive - depart).seconds + 86400) / 60

            if minutes > 60:
                continue

            if minutes > 0:
                times[(current['역사명'], next_st['역사명'])] = round(minutes, 1)

        except Exception:
            continue

    return times

In [20]:
segments = {
    "line_1_main": line_1_main,
    "line_1_junction_at_guro_main": line_1_junction_at_guro_main,
    "line_1_junction_at_geumcheon_gu_office": line_1_junction_at_geumcheon_gu_office,
    "line_1_junction_at_byeongjeom": line_1_junction_at_byeongjeom,


    "line_2_in": line_2_in,
    # "line_2_out": line_2_out,
    "line_2_junction_kachisan": line_2_junction_kachisan,
    "line_2_junction_sinsuldong": line_2_junction_sinsuldong,

    "line_3_main": line_3_main,

    "line_4_split_1": line_4_split_1,
    "line_4_split_2": line_4_split_2,

    "line_5_main": line_5_main,
    "line_5_junction": line_5_junction,
}

In [21]:
line_travel_time_dicts = {}

for name, df in segments.items():
    line_travel_time_dicts[name] = extract_train_times(df)

In [22]:
subway_graph = defaultdict(list)
station_lines_map = defaultdict(set)

def add_edges_from_segment(travel_time_dict, line_name):
    for (u, v), w in travel_time_dict.items():
        # 중복제거
        if not any((n == v and ln == line_name) for (n, _, ln) in subway_graph[u]):
            subway_graph[u].append((v, w, line_name))
            subway_graph[v].append((u, w, line_name))
        station_lines_map[u].add(line_name)
        station_lines_map[v].add(line_name)

In [23]:
for line_name, time_dict in line_travel_time_dicts.items():
    add_edges_from_segment(time_dict, line_name)

In [24]:
transfer_stations = {
    station for station, lines in station_lines_map.items()
    if len(lines) > 1
}

In [27]:
len(transfer_stations)

25